### IMDB 영화리뷰 감성분석

In [1]:
import numpy as np
import tensorflow as tf
seed = 2023
np.random.seed(seed)
tf.random.set_seed(seed)

In [2]:
from tensorflow.keras.datasets import imdb
(X_train, y_train), (X_test, y_test) = imdb.load_data()

17464789/17464789 [==============================] - 0s 0us/step


In [3]:
len(X_train), len(X_test)

(25000, 25000)

In [4]:
# 등장한 단어수
len(imdb.get_word_index())

1641221/1641221 [==============================] - 0s 0us/step


88584

In [5]:
print(X_train[0])

[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 22665, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 21631, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 19193, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 10311, 8, 4, 107, 117, 5952, 15, 256, 4, 31050, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 12118, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]


In [6]:
# 리뷰 최대/평균 길이
max(len(s) for s in X_train), sum(len(s) for s in X_train) / len(X_train)

(2494, 238.71364)

#### 1. LSTM으로 감성분석
- 단어빈도수 기준 많이 사용되는 10,000 단어만 사용
- 한 문장의 단어수 : 200 단어 / 두괄식으로 표현

In [7]:
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [8]:
num_words = 10000
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=num_words)

In [9]:
max_len = 200
X_train = pad_sequences(X_train, maxlen=max_len)
X_test = pad_sequences(X_test, maxlen=max_len)
X_train.shape, X_test.shape

((25000, 200), (25000, 200))

- 모델
    - Embedding vector size : 100 ea
    - LSTM node 수 : 128 units

In [10]:
model = Sequential([
    Embedding(num_words, 100, input_length=max_len),
    LSTM(128),      # activation='tanh' 생략
    Dense(1, activation='sigmoid')
])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 200, 100)          1000000   
                                                                 
 lstm (LSTM)                 (None, 128)               117248    
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1117377 (4.26 MB)
Trainable params: 1117377 (4.26 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [11]:
model.compile('adam', 'binary_crossentropy', ['accuracy'])

model_path = 'best-imdb-lstm.h5'
mc = ModelCheckpoint(model_path, save_best_only=True, verbose=1)
es = EarlyStopping(patience=5)

In [12]:
hist = model.fit(
    X_train, y_train, validation_split=0.2,
    epochs=30, batch_size=64, callbacks=[mc, es]
)

Epoch 1/30
313/313 [==============================] - ETA: 0s - loss: 0.4174 - accuracy: 0.7981
Epoch 1: val_loss improved from inf to 0.32181, saving model to best-imdb-lstm.h5
313/313 [==============================] - 48s 131ms/step - loss: 0.4174 - accuracy: 0.7981 - val_loss: 0.3218 - val_accuracy: 0.8678
Epoch 2/30
  1/313 [..............................] - ETA: 4s - loss: 0.2916 - accuracy: 0.8906

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


313/313 [==============================] - ETA: 0s - loss: 0.2647 - accuracy: 0.8957
Epoch 2: val_loss did not improve from 0.32181
313/313 [==============================] - 18s 56ms/step - loss: 0.2647 - accuracy: 0.8957 - val_loss: 0.3758 - val_accuracy: 0.8662
Epoch 3/30
313/313 [==============================] - ETA: 0s - loss: 0.1745 - accuracy: 0.9376
Epoch 3: val_loss did not improve from 0.32181
313/313 [==============================] - 10s 32ms/step - loss: 0.1745 - accuracy: 0.9376 - val_loss: 0.3398 - val_accuracy: 0.8734
Epoch 4/30
313/313 [==============================] - ETA: 0s - loss: 0.1254 - accuracy: 0.9547
Epoch 4: val_loss did not improve from 0.32181
313/313 [==============================] - 8s 26ms/step - loss: 0.1254 - accuracy: 0.9547 - val_loss: 0.4013 - val_accuracy: 0.8656
Epoch 5/30
313/313 [==============================] - ETA: 0s - loss: 0.0907 - accuracy: 0.9685
Epoch 5: val_loss did not improve from 0.32181
313/313 [==============================] 

In [13]:
best_model = load_model(model_path)
best_model.evaluate(X_test, y_test)

782/782 [==============================] - 4s 5ms/step - loss: 0.3393 - accuracy: 0.8549


[0.339260458946228, 0.8548799753189087]

#### 2. Conv1D로 학습

In [14]:
from tensorflow.keras.layers import Conv1D, MaxPooling1D, GlobalMaxPooling1D, Dropout

In [15]:
model2 = Sequential([
    Embedding(num_words, 100, input_length=max_len),
    Dropout(0.5),
    Conv1D(64, 7, activation='relu'),       # (7,)의 벡터가 생성
    MaxPooling1D(7),
    Conv1D(64, 5, activation='relu'),
    MaxPooling1D(5),
    GlobalMaxPooling1D(),       # Flatten()과 같은 역할
    Dense(1, 'sigmoid')
])

In [17]:
model2.compile('adam', 'binary_crossentropy', ['accuracy'])

model_path2 = 'best-imdb-conv1D.h5'
mc2 = ModelCheckpoint(model_path2, save_best_only=True, verbose=1)
es2 = EarlyStopping(patience=5)

In [18]:
hist2 = model2.fit(
    X_train, y_train, validation_split=0.2,
    epochs=30, batch_size=64, callbacks=[mc2, es2]
)

Epoch 1/30
313/313 [==============================] - ETA: 0s - loss: 0.5240 - accuracy: 0.7099
Epoch 1: val_loss improved from inf to 0.36440, saving model to best-imdb-conv1D.h5
313/313 [==============================] - 34s 81ms/step - loss: 0.5240 - accuracy: 0.7099 - val_loss: 0.3644 - val_accuracy: 0.8456
Epoch 2/30
  3/313 [..............................] - ETA: 17s - loss: 0.3313 - accuracy: 0.8490

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


313/313 [==============================] - ETA: 0s - loss: 0.3003 - accuracy: 0.8722
Epoch 2: val_loss improved from 0.36440 to 0.35418, saving model to best-imdb-conv1D.h5
313/313 [==============================] - 10s 33ms/step - loss: 0.3003 - accuracy: 0.8722 - val_loss: 0.3542 - val_accuracy: 0.8498
Epoch 3/30
313/313 [==============================] - ETA: 0s - loss: 0.2149 - accuracy: 0.9167
Epoch 3: val_loss did not improve from 0.35418
313/313 [==============================] - 6s 20ms/step - loss: 0.2149 - accuracy: 0.9167 - val_loss: 0.3721 - val_accuracy: 0.8506
Epoch 4/30
313/313 [==============================] - ETA: 0s - loss: 0.1565 - accuracy: 0.9401
Epoch 4: val_loss did not improve from 0.35418
313/313 [==============================] - 5s 15ms/step - loss: 0.1565 - accuracy: 0.9401 - val_loss: 0.4178 - val_accuracy: 0.8492
Epoch 5/30
313/313 [==============================] - ETA: 0s - loss: 0.1108 - accuracy: 0.9583
Epoch 5: val_loss did not improve from 0.35418
3

In [19]:
best_model2 = load_model(model_path2)
best_model2.evaluate(X_test, y_test)

782/782 [==============================] - 3s 3ms/step - loss: 0.3732 - accuracy: 0.8332


[0.3731740117073059, 0.8331599831581116]